In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import skimage.measure

just - mean may effective for special case of images.

But it reduce model's performance in my case.

In [ ]:
train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df['path'] = train_df.id.apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_df.head()

In [ ]:
def sig(x):
    # for easy EDA.
    return 1/(1+np.exp(-x*1.5))

In [ ]:
def plot(img, target, file_id):
    _, w, h = img.shape
    img_ = np.vstack([img[0],img[2],img[4]]).transpose()
    w1, h1 = img_.shape
    
    fig = plt.figure(figsize=(14,10))
    fig.suptitle(f"{file_id}.npy, target: {target}")
    plt.subplot(2,2,1)
    plt.title('original')
    plt.imshow(sig(img_), aspect='auto')
    
    img0 = img[0].mean(axis=0)
    img0 = cv2.resize(img0, (w,h)).transpose()
    img2 = img[2].mean(axis=0)
    img2 = cv2.resize(img2, (w,h)).transpose()
    img4 = img[4].mean(axis=0)
    img4 = cv2.resize(img4, (w,h)).transpose()
    
    img0 = img[0]-img0
    img2 = img[2]-img2
    img4 = img[4]-img4
    img0_ = img0.mean(axis=1)
    img0_ = cv2.resize(img0_, (h,w))
    img2_ = img2.mean(axis=1)
    img2_ = cv2.resize(img2_, (h,w))
    img4_ = img4.mean(axis=1)
    img4_ = cv2.resize(img4_, (h,w))
    img0 = img0 - img0_
    img2 = img2 - img2_
    img4 = img4 - img4_
    img = np.vstack([img0,img2,img4]).transpose()
    plt.subplot(2,2,2)
    plt.title('-mean')
    plt.imshow(sig(img), aspect='auto')
    
    
    img_2 = skimage.measure.block_reduce(img_, (5,5), np.mean)
    img_2 = cv2.resize(img_2, (h1, w1))
    plt.subplot(2,2,3)
    plt.title('-mean_pool')
    plt.imshow(sig(img_ - img_2), aspect='auto')

    plt.subplot(2,2,4)
    plt.title('mean_pool')
    plt.imshow(sig(img_2), aspect='auto')
    plt.show()

In [ ]:
trn_df = pd.concat([train_df[train_df.target==0].sample(10), train_df[train_df.target==1].sample(30)]).sample(40).reset_index(drop=True)

In [ ]:
for idx in range(len(trn_df)):
    file_id, target, path = trn_df.iloc[idx]
    img = np.load(path).astype('float')
    
    plot(img, target, file_id)

meanpool < original + meanpool < original - mean < original - meanpool